In [1]:
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [4]:
from hypers import *
from utils import *
from generator import *
from models import *
from preprocessing import *

In [5]:
data_root_path = None
embedding_path = None
KG_root_path = None

In [6]:
news,news_index,category_dict,subcategory_dict,word_dict = read_news(data_root_path,'docs.tsv')
news_title,news_vert,news_subvert=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict)
title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)

In [7]:
graph, EntityId2Index, EntityIndex2Id, entity_embedding = load_entity_metadata(KG_root_path)
news_entity = load_news_entity(news,EntityId2Index,data_root_path)

In [8]:
news_entity_index = parse_zero_hop_entity(EntityId2Index,news_entity,news_index,max_entity_num)
one_hop_entity = parse_one_hop_entity(EntityId2Index,EntityIndex2Id,news_entity_index,graph,news_index,max_entity_num)

In [9]:
train_session = read_clickhistory(data_root_path,news_index,'train_sam2.tsv')
train_user = parse_user(news_index,train_session)
train_sess, train_user_id, train_label = get_train_input(news_index,train_session)
train_generator = get_hir_train_generator(news_title,news_entity_index,one_hop_entity,entity_embedding,train_user['click'],train_user_id,train_sess,train_label,16)

In [10]:
test_session = read_clickhistory(data_root_path,news_index,'val_sam2.tsv')
test_user = parse_user(news_index,test_session)
test_docids, test_userids, test_labels, test_bound = get_test_input(news_index,test_session)
test_generator = get_test_generator(test_docids,test_userids,news_title,news_entity_index,one_hop_entity,entity_embedding,test_user['click'],64)

In [ ]:
EPOCH = 7
model,inter_model = create_model(title_word_embedding_matrix)
model.fit_generator(train_generator,epochs=EPOCH)
predicted_label = inter_model.predict_generator(test_generator,verbose=1)
result = evaluate(predicted_label,test_labels,test_bound)

Epoch 1/3
14772/14772 [==============================] - 12696s 859ms/step - loss: 1.3966 - acc: 0.4195
Epoch 2/3
14772/14772 [==============================] - 12898s 873ms/step - loss: 1.3313 - acc: 0.4568
Epoch 3/3
 5330/14772 [=========>....................] - ETA: 2:20:40 - loss: 1.3043 - acc: 0.4707